```
Attention Mechanism

고양이가 잔다 - > the cat sleeps

입력 : "고양이가 잔다"

[전처리] -> <start> 고양이가 잔다 <end>

[ENCODER] -> 각 단어마다 hidden state 생성

[ATTENTION] -> 어디에 집중할지 계산

[DECODER] -> 한 단어씩 생성

출력 : the cat sleeps
```



In [ ]:
# 원본
입력 = "고양이가 자요"
# 1단계 전처리
전처리_후 = f'<start> {input} <end>'
# 2단계 토큰화(단어->숫자)
단어_사전 = {
    '<pad>' : 0,
    '<start>' : 1,
    '<end>' : 2,
    '고양이가' : 3,
    '자요' : 4
}
# 최대길이 5대로 
정수_시퀀스 = [1,3,4,2,0]

In [ ]:
# STEP 1 : EMBEDDING(숫자 -> 벡터)
# 임베딩레이어 =========== 256차원
정수_시퀀스 = [1,3,4,2,0]
# Embedding.....
임베딩_결과 = [
    [0.2,-0.5, ..., 0.1]   #1 (<start>)의 벡터 (265)차원
    ,[0.2,-0.5, ..., 0.3]  #3 (고양이가)의 벡터 (265)차원
    ,[0.2,-0.5, ..., 0.4]  #4 (자요)의 벡터 (265)차원
    ,[0.2,-0.5, ..., 0.2]  #2 (<end>)의 벡터 (265)차원
    ,[0.0,0.0, ..., 0.0]  #0 (<pad>)의 벡터 (265)차원
]

In [ ]:
import tensorflow as tf
LSTM = tf.keras.layer.LSTN()
# STEP 2 : ENCODER(벡터-> Hidden States)
# LSTM Encoder
# 각 타임스텝마다 hidden state 생성

# 초기상태
h0 =[0,0,0, ..., 0] # 512차원 영벡터 
c0 =[0,0,0, ..., 0] # 512차원 영벡터 

# ============= 타임스텝 1 : <start> 처리 ================
입력_t1 = [0.2,-0.5, ..., 0.1] # <start> 의 임베딩
h1, c1 =  LSTM(입력_t1, h0, c0)
# h1 = [0.1 .....] 512차원

# ============= 타임스텝 2 : 고양이가 처리 ================
입력_t2 = [0.2,-0.5, ..., 0.3] # 고양이가 의 임베딩
h2, c2 =  LSTM(입력_t2, h1, c1)

# ============= 타임스텝 3 : 자요 처리 ================
입력_t3 = [0.2,-0.5, ..., 0.4] # 자요 의 임베딩
h3, c3 =  LSTM(입력_t3, h2, c2)

# ============= 타임스텝 4 : <end> 처리 ================
입력_t4 = [0.2,-0.5, ..., 0.2] # <end> 의 임베딩
h4, c4 =  LSTM(입력_t4, h3, c3)

# ============= 타임스텝 4 : 패딩 (무시) ================
입력_t5 = [0.0,0.0, ..., 0.0]
h5, c5 =  LSTM(입력_t5, h4, c4)
# h5는 사용 안함(마스킹)

# ============ 인코더의 출력 =================
encoder_output = [h1,h2,h3,h4,h5] # 모든 hidden states
# 형태 [1,5,512]   [배치, 시퀀스,units]
encoder_final_h = h4  # 마지막 hidden state(디코더 초기화용)
encoder_final_c = c4  # 마지막 cell state

# ================= 정리 ==================
# <start>   ----> h1
# 고양이가   --->   h2    "고양이"  정보를 압축
# 자요       --->   h3    "자요"  정보를 압축
# <end>      --->  h4     전체 문장 요약
# <pad>      --->  h5   무시됨